------------------------------------------- 
A* implementaion for PS10 by group no. 250
-------------------------------------------



| S. No. | Name        | ID         | Contribution % |
|--------|-------------|------------|----------------|
| 1      | Rahul       | 2024ad05284| 100            |
| 2      | Brijesh     | 2024ad05270| 100            |
| 3      | Mandan      | 2024ad05331| 100            |
| 4      | Gayathri    | 2024ad05359| 100            |
| 5      | Pranav Deep | 2024ad05376| 100            |

In [3]:
import heapq

# Grid size
rows , cols = 6, 7

# start and goal location
start, goal = (3,0), (3,6)

# Movement direction : East, west, north, south
movement = [(1,0), (-1,0), (0,-1), (0,1)]

# cost
cell_move_cost = 3
additional_east_move_cost = 5


In [ ]:
# Define walls position


In [ ]:
# Heuristic function
def heuristic_caclc (g, h, w):
    return g+w*h